# Lab: Building Simple RAG Using Langchain

In this lab, we will build a **Retrieval-Augmented Generation (RAG)** system that combines information retrieval and text generation to process large documents, such as legal contracts. The RAG system leverages **FAISS** for efficient similarity search and **Hugging Face's generative models** (like GPT-2) for synthesizing human-like answers. By embedding the document content and indexing it with FAISS, we can quickly retrieve relevant sections of text and generate contextually accurate responses to specific queries.

*The main objective of this system is to enable automated document analysis, allowing users to query large legal documents and get detailed, relevant, and concise answers without manually reading through the entire document.*

This approach significantly improves efficiency for industries like law, where processing vast amounts of textual data is crucial.

# Step 1: Install Required Libraries

In this step, we install the necessary libraries that will be used throughout the lab. **LangChain** is essential for orchestrating the RAG system, integrating document processing and retrieval with language models.

**FAISS** is used to index and search the document embeddings efficiently, which allows us to quickly retrieve relevant content. **Transformers** provides access to pre-trained language models, and **Sentence-Transformers** allows us to generate meaningful sentence embeddings.

 These libraries are fundamental to building a scalable and efficient RAG system.

In [ ]:
!pip install langchain
!pip install faiss-cpu
!pip install transformers
!pip install sentence-transformers


#### **Explanation of the Code:**

These commands install essential libraries needed to build a Retrieval-Augmented Generation (RAG) system using LangChain and related tools.

* **!pip install langchain:** Installs LangChain, a framework for integrating large language models (LLMs) with tasks like document retrieval and generation.

* **!pip install faiss-cpu:** Installs FAISS, a library for fast similarity search over high-dimensional data, used for efficient document retrieval.

* **!pip install transformers:** Installs the Transformers library, providing access to pre-trained models for tasks like text generation and embeddings.

* **!pip install sentence-transformers:** Installs Sentence-Transformers, which generates sentence embeddings to capture the semantic meaning of text for similarity-based tasks.

# Step 2: Import Required Libraries

1. The commands update **LangChain** to its latest version and install the **LangChain Community edition**, which provides additional features and integrations for building AI applications like document retrieval and text generation.

In [ ]:
!pip install --upgrade langchain
!pip install langchain-community


#### **Explanation of the Code:**

* **!pip install --upgrade langchain:** Upgrades the LangChain library to the latest version, ensuring you have access to the newest features, improvements, and bug fixes for developing applications with large language models (LLMs).

* **!pip install langchain-community:** Installs the LangChain Community edition, which includes additional tools, integrations, and community-driven enhancements that extend the core LangChain functionality for more advanced use cases.

2. This code imports libraries for building a **RAG** system, including **LangChain** for document retrieval and generation, FAISS for similarity search, and **SentenceTransformer** for generating embeddings. It also uses **TextLoader** to load documents and **RecursiveCharacterTextSplitter** to split large texts into chunks for efficient processing.

In [ ]:
import os
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer


The commands themselves don't produce direct outputs, as they are imports. However, the expected behavior is:

* The necessary modules and functions are loaded into the environment, making them available for use in the code.

* No explicit output will be displayed unless errors occur (e.g., if a module is not installed or there is an issue with the import).

#### **Explanation of the Code:**

1. **import os**: Imports the os module for interacting with the operating system, such as file handling and directory management.

2. **from langchain.chains import RetrievalQA**: Imports RetrievalQA from LangChain, enabling the creation of a chain that combines document retrieval and question answering.

3. **from langchain.vectorstores import FAISS**: Imports FAISS from LangChain, which is used to create a vector store for efficient similarity search over high-dimensional data.

4. **from langchain.embeddings import HuggingFaceEmbeddings**: Imports HuggingFaceEmbeddings from LangChain, enabling the generation of text embeddings using pre-trained models from Hugging Face.

5. **from langchain.document_loaders import TextLoader**: Imports TextLoader from LangChain to load text documents (e.g., .txt files) for further processing.

6. **from langchain.text_splitter import RecursiveCharacterTextSplitter** :Imports RecursiveCharacterTextSplitter from LangChain, used for splitting large documents into smaller, manageable chunks based on character count.

7. **from sentence_transformers import SentenceTransformer**: Imports SentenceTransformer, a library for generating sentence embeddings to represent the semantic meaning of text.

3. To verify that **LangChain** and its dependencies are installed correctly, you can run the below code. This will **print the version** of LangChain you have installed. Ensure it is up to date *(e.g., version 0.3.x or higher).*

In [ ]:
import langchain
print(langchain.__version__)


#### **Expected Output:**

The version number of the LangChain library installed in your environment will be printed

```0.3.27```

# Step 3: Load and Preprocess Data (Documents)


1. In this step, the **document** (such as a legal contract) is uploaded to Google Colab using the files.upload() function, which opens a dialog for selecting the file. This is essential to bring the document into the environment for further processing, and it ensures that the file is **accessible** for loading and manipulation.

Here, we will assume you have a text file with your legal documents. If you Not then **Download the Document form Here:**

[legal_documents.txt](https://github.com/k21academyuk/Agentic-AI/blob/main/legal_documents.txt)

In [ ]:
from google.colab import files
uploaded = files.upload()  # This will open the file upload dialog


#### **Code Explanation:**

1. **from google.colab import files**: This imports the files module from Google Colab, which provides methods for file handling in a Colab environment, such as uploading or downloading files.

2. **uploaded = files.upload():** This triggers the file upload dialog in Google Colab, allowing the user to select and upload files from their local system to the Colab environment. The uploaded files are stored in the uploaded variable, which is a dictionary containing the uploaded file names and their corresponding file objects.

2. After uploading the file, you can **verify the file's existence** using the os module:


In [ ]:
import os
print(os.listdir())  # This will show the files in the current directory


#### **Expected Output:**

The output will be a list of files and directories in the current directory. For example:

```['.config', 'legal_documents.txt', 'sample_data']```

This shows the files available in the current directory, including the uploaded file ```legal_documents.txt.```

#### **Code Explanation:**

* **print(os.listdir())**: This prints the list of files and directories present in the current working directory. It helps verify the available files in the environment.

3. The code uses ```TextLoader``` from LangChain to load the **legal document** ```(legal_documents.txt)```. The ```load()``` method reads the document and prepares it for further processing, such as splitting into chunks or generating embeddings. It is important to ensure the **file name** matches the uploaded document to avoid errors.

In [ ]:
loader = TextLoader("legal_documents.txt")  # Make sure the file name matches the uploaded file
documents = loader.load()


#### **Code Explanation:**

* **loader = TextLoader("legal_documents.txt"):** This initializes a TextLoader object that loads the legal_documents.txt file from the current directory. It's crucial that the file name matches exactly, including the extension, to avoid errors.

* **documents = loader.load():** The load() method reads the content of the legal_documents.txt file and stores it in the documents variable, making the text available for further processing.

4. After loading the document, we use the ```RecursiveCharacterTextSplitter``` to break the document into smaller, manageable chunks of text. Each chunk is typically sized at 1000 characters with an overlap of 200 characters. This step is important for **handling large documents effectively**, ensuring that chunks maintain context while being small enough for efficient processing, such as embedding generation and retrieval tasks.

In [ ]:
# Load documents (text file in this case)
loader = TextLoader("legal_documents.txt")  # Replace with your actual file path
documents = loader.load()

# Split the documents into chunks to manage large documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)


#### **Code Explanation:**

* The **TextLoader** loads the legal_documents.txt file from the file system, ensuring the file path is correct for successful loading.

* **loader.load()** reads the content of the file and loads it into memory, returning the document in a list format.

* The **RecursiveCharacterTextSplitter** splits the document into smaller chunks of 1000 characters each, with a 200-character overlap to ensure context is maintained between adjacent chunks.

* **split_documents(documents)** processes the loaded document and divides it into manageable text chunks for more efficient processing, particularly useful for tasks like embedding or retrieval.

# Step 4: Generate Embeddings with Hugging Face

In this step, we initialize a **Sentence Transformer model** from Hugging Face, specifically the ```all-MiniLM-L6-v2``` model, which is pre-trained to generate high-quality **sentence embeddings**. These embeddings are vector representations of text that capture the semantic meaning of each document chunk. The model is applied to each chunk of text from the previously split document, and the ```encode()``` method generates an embedding for each chunk. By creating embeddings, we can efficiently compare the semantic similarity between text chunks, which is crucial for **document retrieval** and generating contextually relevant answers to user queries.

In [ ]:
# Initialize the Hugging Face model (using a sentence transformer model)
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

# Generate embeddings for each document chunk
embeddings = [embedding_model.encode(text.page_content) for text in texts]  # Access page_content for text in documents

# Now 'embeddings' will contain the embeddings for each chunk of text in the document


#### **Code Explanation:**

* **SentenceTransformer('all-MiniLM-L6-v2')** initializes the Hugging Face model all-MiniLM-L6-v2, which is a pre-trained model designed to generate embeddings that capture the semantic meaning of text.

* **embedding_model.encode(text.page_content)** generates embeddings for each document chunk. The encode() function processes the text chunks and converts them into vector representations (numerical embeddings).

* The result is a list of embeddings, where each entry corresponds to a chunk of text, allowing for efficient comparison and retrieval based on semantic similarity.

# Step 5: Create FAISS Vector Store

1. This step creates a **FAISS vector store** to store the embeddings generated from document chunks. FAISS enables efficient similarity search, allowing fast retrieval of relevant text chunks based on a query. The **HuggingFaceEmbeddings** class from LangChain generates these embeddings. However, there's a deprecation warning suggesting the use of the updated ```langchain_huggingface``` package for future compatibility.

In [ ]:
from langchain.vectorstores import FAISS

# Create a FAISS vector store using the embeddings
vector_store = FAISS.from_documents(texts, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))


#### **Code Explanation:**

* ```from langchain.vectorstores import FAISS:``` This imports the FAISS class from LangChain, which is used to create an efficient similarity search index for document embeddings.

* ```FAISS.from_documents(texts, HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2"))```: This line creates a **FAISS vector** store using the document chunks (stored in texts) and the **HuggingFaceEmbeddings** to generate embeddings from the sentence-transformers/all-MiniLM-L6-v2 model. The vector store allows for fast similarity searches on the document embeddings.

2. In this step the **FAISS index** is then saved locally with ```save_local()``` for future use, avoiding the need to regenerate the index every time.

In [ ]:
# Save the FAISS index (optional, for future use)
vector_store.save_local("faiss_index")


This command **saves** the FAISS vector store locally to a directory named ```faiss_index```. Saving the index allows you to reuse it later without needing to regenerate the embeddings, saving computational resources and time. This is especially useful for large datasets or when you want to store the index for future retrieval operations.

# Step 6: Set Up Retriever and Generative Model

In this step, we set up the **retriever** to fetch the most relevant chunks of text from the **FAISS vector store** based on the user’s query. We also initialize a **text generation model** from **Hugging Face** (e.g., GPT-2) to synthesize answers based on the retrieved text. The **retriever** fetches the top **k** relevant document chunks, while the **generative model** generates a concise response based on those chunks. Combining these two components, the **RetrieverQA** chain enables the RAG system to retrieve context and generate a comprehensive answer, automating document analysis.

1. This step sets up a **retriever** and a generative model using LangChain and Hugging Face, enabling you to build a system that retrieves relevant information from documents and generates human-like responses.

In [ ]:
# Import libraries
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

#### **Code Explanation:**

* ```from langchain.llms import HuggingFacePipeline```: This imports the **HuggingFacePipeline** class from LangChain, which is used to integrate Hugging Face models into LangChain workflows. It allows you to use Hugging Face's pre-trained models (like GPT) for tasks such as text generation within LangChain chains.

* ```from transformers import pipeline```:This imports the pipeline function from the Transformers library. The pipeline function is used to load and use pre-trained models for specific NLP tasks such as text generation, question answering, translation, etc.

2. This step sets up the retriever to fetch relevant document chunks based on a query and initializes a **text generation model** to generate responses from those chunks.

In [ ]:
# Set up the retriever to fetch the top 2 most relevant chunks for each query
retriever = vector_store.as_retriever(search_kwargs={"k": 2})

# Use Hugging Face's GPT model for text generation
generator = pipeline('text-generation', model='gpt2')  # You can change the model (e.g., GPT-Neo)

#### **Code Explanation:**

* **Retriever** fetches the top 2 relevant document chunks based on the query.

* GPT-2 is set up as a **generative model** to produce responses based on the retrieved chunks.

3. This step integrates the retriever and **text generation model (LLM)** into a single **LangChain QA (Question-Answering) system.**

In [ ]:
# Wrap Hugging Face pipeline in LangChain's HuggingFacePipeline
# Added max_length to limit the input sequence length
llm = HuggingFacePipeline(pipeline=generator, model_kwargs={'max_length': 512})

# Combine the retriever and LLM into a single chain
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

#### **Code Explanation:**

* The **HuggingFacePipeline** wraps the Hugging Face model for easy use in LangChain.

* ```llm=llm``` connects the Hugging Face model (GPT-2) as the generative model for the QA process.

* ```retriever=retriever``` links the retriever (which fetches relevant document chunks) to the QA chain.

* ```chain_type="stuff"``` indicates that the system will use the full retrieved context (document chunks) for generating answers.

* **RetrievalQA** combines the retriever and LLM into a single QA chain, enabling automated answers based on the context retrieved from the documents.

# Step 7: Query the System

In this step, we query the system (e.g., asking about termination clauses in a contract). The system retrieves relevant chunks of text and generates an answer using the pre-configured **retriever** and **generative model** (GPT-2). This step shows how the system can automatically generate responses based on specific queries.



In [ ]:
# Query the system for termination clauses
query = "What are the termination clauses in this contract?"
result = qa.run(query)

# Print the result (the answer generated by the system)
print(result)


#### **Code Explanation:**

* ```result = qa.run(query)```: This line runs the QA chain (qa), which has already been set up with a retriever and a generative model (likely GPT-based model).

The system should ideally return a concise answer that directly addresses the termination clauses of the contract.